## Nobel API - check Wikidata and Nobel API
version. 0.2 changed api call and added **Vrandečić EWS**
* This [Jupyter Notebook](https://github.com/salgo60/open-data-examples/blob/master/Nobel%20API.ipynb)  

  * WIkidata property [Property:P8024](https://www.wikidata.org/wiki/Property:P8024)
  * [Developer xone nobelprize.org](https://www.nobelprize.org/about/developer-zone-2/)
      * maybe better API is [api.nobelprize.org/2.0/laureates](https://api.nobelprize.org/2.0/laureates)

In [15]:
from datetime import datetime
start_time = datetime.now()
print("Last run: ", datetime.now())

Last run:  2020-10-07 00:12:41.845551


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 

url ="https://api.nobelprize.org/2.0/laureates?offset=0&limit=1500"
dftot = pd.DataFrame()
r = http.request('GET', url)
data = json.loads(r.data)
dftot = dftot.append(pd.DataFrame(data["laureates"]),sort=False)

In [3]:
dftot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 949 entries, 0 to 948
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               949 non-null    object
 1   knownName        925 non-null    object
 2   givenName        925 non-null    object
 3   familyName       923 non-null    object
 4   fullName         925 non-null    object
 5   gender           925 non-null    object
 6   birth            924 non-null    object
 7   links            949 non-null    object
 8   nobelPrizes      949 non-null    object
 9   death            628 non-null    object
 10  orgName          24 non-null     object
 11  nativeName       24 non-null     object
 12  acronym          10 non-null     object
 13  founded          23 non-null     object
 14  penName          11 non-null     object
 15  birthCountry     1 non-null      object
 16  birthCountryNow  1 non-null      object
 17  birthContinent   1 non-null      ob

In [4]:
dftot.tail(10)


,id,knownName,givenName,familyName,fullName,gender,birth,links,nobelPrizes,death,orgName,nativeName,acronym,founded,penName,birthCountry,birthCountryNow,birthContinent
939,50,"{'en': 'Wolfgang Pauli', 'se': 'Wolfgang Pauli'}","{'en': 'Wolfgang', 'se': 'Wolfgang'}","{'en': 'Pauli', 'se': 'Pauli'}","{'en': 'Wolfgang Pauli', 'se': 'Wolfgang Pauli'}",male,"{'date': '1900-04-25', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '1945', 'category': {'en': 'Phy...","{'date': '1958-12-15', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,483,"{'en': 'Woodrow Wilson', 'se': 'Woodrow Wilson'}","{'en': 'Woodrow', 'se': 'Woodrow'}","{'en': 'Wilson', 'se': 'Wilson'}","{'en': 'Thomas Woodrow Wilson', 'se': 'Thomas ...",male,"{'date': '1856-12-28', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '1919', 'category': {'en': 'Pea...","{'date': '1924-02-03', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,557,"{'en': 'Yasser Arafat', 'se': 'Yasser Arafat'}","{'en': 'Yasser', 'se': 'Yasser'}","{'en': 'Arafat', 'se': 'Arafat'}","{'en': 'Yasser Arafat', 'se': 'Yasser Arafat'}",male,"{'date': '1929-08-24', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '1994', 'category': {'en': 'Pea...","{'date': '2004-11-11', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
942,642,"{'en': 'Yasunari Kawabata', 'se': 'Yasunari Ka...","{'en': 'Yasunari', 'se': 'Yasunari'}","{'en': 'Kawabata', 'se': 'Kawabata'}","{'en': 'Yasunari Kawabata', 'se': 'Yasunari Ka...",male,"{'date': '1899-06-11', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '1968', 'category': {'en': 'Lit...","{'date': '1972-04-16', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
943,559,"{'en': 'Yitzhak Rabin', 'se': 'Yitzhak Rabin'}","{'en': 'Yitzhak', 'se': 'Yitzhak'}","{'en': 'Rabin', 'se': 'Rabin'}","{'en': 'Yitzhak Rabin', 'se': 'Yitzhak Rabin'}",male,"{'date': '1922-03-01', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '1994', 'category': {'en': 'Pea...","{'date': '1995-11-04', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
944,826,"{'en': 'Yoichiro Nambu', 'se': 'Yoichiro Nambu'}","{'en': 'Yoichiro', 'se': 'Yoichiro'}","{'en': 'Nambu', 'se': 'Nambu'}","{'en': 'Yoichiro Nambu', 'se': 'Yoichiro Nambu'}",male,"{'date': '1921-01-18', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '2008', 'category': {'en': 'Phy...","{'date': '2015-07-05', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
945,927,"{'en': 'Yoshinori Ohsumi', 'se': 'Yoshinori Oh...","{'en': 'Yoshinori', 'se': 'Yoshinori'}","{'en': 'Ohsumi', 'se': 'Ohsumi'}","{'en': 'Yoshinori Ohsumi', 'se': 'Yoshinori Oh...",male,"{'date': '1945-02-09', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '2016', 'category': {'en': 'Phy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
946,265,"{'en': 'Yuan T. Lee', 'se': 'Yuan T. Lee'}","{'en': 'Yuan T.', 'se': 'Yuan T.'}","{'en': 'Lee', 'se': 'Lee'}","{'en': 'Yuan T. Lee', 'se': 'Yuan T. Lee'}",male,"{'date': '1936-11-19', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '1986', 'category': {'en': 'Che...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
947,794,"{'en': 'Yves Chauvin', 'se': 'Yves Chauvin'}","{'en': 'Yves', 'se': 'Yves'}","{'en': 'Chauvin', 'se': 'Chauvin'}","{'en': 'Yves Chauvin', 'se': 'Yves Chauvin'}",male,"{'date': '1930-10-10', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '2005', 'category': {'en': 'Che...","{'date': '2015-01-27', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
948,726,"{'en': 'Zhores Alferov', 'se': 'Zhores Alferov'}","{'en': 'Zhores', 'se': 'Zhores'}","{'en': 'Alferov', 

In [5]:
listNobel = []
for index,row in dftot.iterrows():
    new_item = dict()
    new_item['id'] = row['id']
    try:
        new_item['name_en'] = row['knownName']['en']
    except:
        pass
    try:
        new_item['name_se'] = row['knownName']['se']
    except:
        pass
    new_item['link'] = row['links']['href']
    try:
        new_item['gender'] = row['gender']
    except:
        pass
    listNobel.append(new_item)
NobelTot = pd.DataFrame(listNobel,
                  columns=['id','name_en','name_se','link','gender'])

NobelTot.info()    


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 949 entries, 0 to 948
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       949 non-null    object
 1   name_en  925 non-null    object
 2   name_se  925 non-null    object
 3   link     949 non-null    object
 4   gender   925 non-null    object
dtypes: object(5)
memory usage: 37.2+ KB


## EWS Vrandečić

In [6]:
NobelTot[NobelTot['name_en'].str.contains("Vrandečić", na=False)] 

,id,name_en,name_se,link,gender


In [7]:
#check number of prizes received
NobelTot[NobelTot['name_en'].str.contains("Vrandečić", na=False)]["name_en"].sum()


0

## Check Wikidata matches

In [8]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?item ?itemLabel ?NobelAPI WHERE {
  ?item wdt:P8024 ?NobelAPI.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} order by xsd:integer(?NobelAPI)"""


def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']
    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDNobel = get_sparql_dataframe(endpoint_url, query)

In [9]:
WDNobel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947 entries, 0 to 946
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item       947 non-null    object
 1   itemLabel  947 non-null    object
 2   NobelAPI   947 non-null    object
dtypes: object(3)
memory usage: 22.3+ KB


### Compare WD <-> Nobel 

In [10]:
dfmerge = pd.merge(WDNobel, NobelTot,how='outer', left_on="NobelAPI",right_on="id",indicator=True)
dfmerge['_merge'] = dfmerge['_merge'].str.replace('left_only','WD_only').str.replace('right_only','Nobel_only')
dfmerge.rename(columns={"_merge": "WD_Nobel"},inplace = True)    
dfmerge[0:10]  

,item,itemLabel,NobelAPI,id,name_en,name_se,link,gender,WD_Nobel
0,http://www.wikidata.org/entity/Q35149,Wilhelm Röntgen,1,1,Wilhelm Conrad Röntgen,Wilhelm Conrad Röntgen,http://masterdataapi.nobelprize.org/2/laureate/1,male,both
1,http://www.wikidata.org/entity/Q41688,Hendrik Lorentz,2,2,Hendrik A. Lorentz,Hendrik A. Lorentz,http://masterdataapi.nobelprize.org/2/laureate/2,male,both
2,http://www.wikidata.org/entity/Q79000,Pieter Zeeman,3,3,Pieter Zeeman,Pieter Zeeman,http://masterdataapi.nobelprize.org/2/laureate/3,male,both
3,http://www.wikidata.org/entity/Q41269,Henri Becquerel,4,4,Henri Becquerel,Henri Becquerel,http://masterdataapi.nobelprize.org/2/laureate/4,male,both
4,http://www.wikidata.org/entity/Q37463,Pierre Curie,5,5,Pierre Curie,Pierre Curie,http://masterdataapi.nobelprize.org/2/laureate/5,male,both
5,http://www.wikidata.org/entity/Q7186,Marie Curie,6,6,Marie Curie,Marie Curie,http://masterdataapi.nobelprize.org/2/laureate/6,female,both
6,http://www.wikidata.org/entity/Q83297,"John William Strutt, 3rd Baron Rayleigh",8,8,Lord Rayleigh,Lord Rayleigh,http://masterdataapi.nobelprize.org/2/laureate/8,male,both
7,http://www.wikidata.org/entity/Q57063,Philipp Lenard,9,9,Philipp Lenard,Philipp Lenard,http://masterdataapi.nobelprize.org/2/laureate/9,male,both
8,http://www.wikidata.org/entity/Q47285,J. J. Thomson,10,10,J.J. Thomson,J.J. Thomson,http://masterdataapi.nobelprize.org/2/laureate/10,male,both
9,http://www.wikidata.org/entity/Q127234,Albert A. Michelson,11,11,Albert A. Michelson,Albert A. Michelson,http://masterdataapi.nobelprize.org/2/laureate/11,male,both


In [11]:
dfmerge["WD_Nobel"].value_counts()  

both          947
Nobel_only      2
Name: WD_Nobel, dtype: int64

In [12]:
Nobelonly = dfmerge[dfmerge["WD_Nobel"]=="Nobel_only"].copy() 

from IPython.display import HTML
Nobelonly["Nobel"] = "<a href='https://api.nobelprize.org/v1/laureate.json?id=" + Nobelonly["id"] + "'>link</a>"
pd.set_option("display.max.columns", None) 
HTML(Nobelonly.to_html(escape=False))

,item,itemLabel,NobelAPI,id,name_en,name_se,link,gender,WD_Nobel,Nobel
947,NaN,NaN,NaN,523,NaN,NaN,http://masterdataapi.nobelprize.org/2/laureate/523,NaN,Nobel_only,link
948,NaN,NaN,NaN,775,Linda B. Buck,Linda B. Buck,http://masterdataapi.nobelprize.org/2/laureate/775,female,Nobel_only,link


In [13]:
print("End run: ", datetime.now())

End run:  2020-10-06 23:58:54.492448


In [16]:
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Time elapsed (hh:mm:ss.ms) 0:00:05.531442
